In [115]:
#Import our libraries
import time
import requests
import json
import xml.etree.ElementTree as ET  # for parsing XML
import xmltodict
import pandas as pd
import csv
import base64 
import sqlalchemy

# Step 1: Get the authentication token and instance key 

In [116]:
#Our endpoint
url="https://service5.ultipro.com/services/BIDataService"

#The below headers variable specifies that we are sending the request to a SOAP API
headers = {"content-type": "application/soap+xml"}

In [117]:
body = """ <s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing">

  <s:Header>

<a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOn</a:Action>

<a:To s:mustUnderstand="1">https://servicehost/services/BiDataService</a:To>

</s:Header>

  <s:Body>

  <LogOn xmlns="http://www.ultipro.com/dataservices/bidata/2">

  <logOnRequest xmlns:i="http://www.w3.org/2001/XMLSchema-instance">

<UserName>BI_Data</UserName>

<Password>SG]xJZ4}/^G!lX!G!#pk5==$HoKo@M|4;O?=$?x3k(|1gd6g_N</Password>

<ClientAccessKey>PLAB4</ClientAccessKey>

<UserAccessKey>ED5P34000010</UserAccessKey>

</logOnRequest>

</LogOn>

</s:Body>

</s:Envelope> """

In [118]:
#View the response
response = requests.post(url, headers=headers, data=body)
print (response)
print (response.text)

<Response [200]>
<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/LogOnResponse</a:Action></s:Header><s:Body><LogOnResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><LogOnResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ServiceId>f0529fed-4f9f-4b5c-b8f8-406268857b3b</ServiceId><ClientAccessKey>PLAB4</ClientAccessKey><Token>3c7c0cf5-9e07-4a60-93cc-5da0ff62c368</Token><Status>Ok</Status><StatusMessage i:nil="true"/><InstanceKey>5e5a22a0-022e-43a8-9f60-eb3d5dfbd4b2</InstanceKey></LogOnResult></LogOnResponse></s:Body></s:Envelope>


Now, iterate through the response to get the token, service_id, and instance key

In [119]:
#Write the response to a file
open("file.xml", "w").write(response.text)

699

In [120]:
#Open the file as an ElementTree object
tree = ET.parse('file.xml')
root = tree.getroot()

In [121]:
#Show all the elements in the tree
[elem.tag for elem in root.iter()]

['{http://www.w3.org/2003/05/soap-envelope}Envelope',
 '{http://www.w3.org/2003/05/soap-envelope}Header',
 '{http://www.w3.org/2005/08/addressing}Action',
 '{http://www.w3.org/2003/05/soap-envelope}Body',
 '{http://www.ultipro.com/dataservices/bidata/2}LogOnResponse',
 '{http://www.ultipro.com/dataservices/bidata/2}LogOnResult',
 '{http://www.ultipro.com/dataservices/bidata/2}ServiceId',
 '{http://www.ultipro.com/dataservices/bidata/2}ClientAccessKey',
 '{http://www.ultipro.com/dataservices/bidata/2}Token',
 '{http://www.ultipro.com/dataservices/bidata/2}Status',
 '{http://www.ultipro.com/dataservices/bidata/2}StatusMessage',
 '{http://www.ultipro.com/dataservices/bidata/2}InstanceKey']

In [122]:
#Get the token from the "Token" element in the tree
for each in root.iter('{http://www.ultipro.com/dataservices/bidata/2}Token'):
    token = each.text

In [123]:
#Get the service id
for each in root.iter('{http://www.ultipro.com/dataservices/bidata/2}ServiceId'):
    service_id = each.text

In [124]:
#Get the instance key
for each in root.iter('{http://www.ultipro.com/dataservices/bidata/2}InstanceKey'):
    instance_key = each.text

In [125]:
#Print the token
print(token)

3c7c0cf5-9e07-4a60-93cc-5da0ff62c368


In [126]:
#Print the service id
print(service_id)

f0529fed-4f9f-4b5c-b8f8-406268857b3b


In [127]:
#Print the instance key
print(instance_key)

5e5a22a0-022e-43a8-9f60-eb3d5dfbd4b2


# Step 2: View the list of reports available

Now we will connect to the GetReportList endpoint, so that we can see the filepaths for all available reports

In [128]:
#Our endpoint
url="https://service5.ultipro.com/services/BIDataService"

#The below headers variable specifies that we are sending the request to a SOAP API
headers = {"content-type": "application/soap+xml"}

In [129]:
body = """   
<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing">

  <s:Header>

<a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/GetReportList</a:Action>

<a:To s:mustUnderstand="1">https://servicehost/services/BiDataService</a:To>

</s:Header>

  <s:Body>

  <GetReportList xmlns="http://www.ultipro.com/dataservices/bidata/2">

  <context xmlns:i="http://www.w3.org/2001/XMLSchema-instance">

<ServiceId>""" + service_id + """</ServiceId>

<ClientAccessKey>PLAB4</ClientAccessKey>

  <!-- GUID, unique per user session -->

<Token>""" + token + """</Token>

<Status>Ok</Status>

<StatusMessage i:nil="true" />

  <!-- GUID, unique per user session -->

<InstanceKey>""" + instance_key + """</InstanceKey>

</context>

</GetReportList>

</s:Body>

</s:Envelope> """

In [130]:
#
response = requests.post(url, headers=headers, data=body)
print (response)
print (response.text)

<Response [200]>
<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/GetReportListResponse</a:Action></s:Header><s:Body><GetReportListResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><GetReportListResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><Reports><Report><ReportName>_Current Employee Deductions and Benefits</ReportName><ReportPath>/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/report[@name='_Current Employee Deductions and Benefits']</ReportPath></Report><Report><ReportName>2013-03-05T21:47:40.188Z</ReportName><ReportPath>/content/folder[@name='UltiPro BI Content']/folder[@name='UltiPro BI for Core HR and Payroll']/folder[@name='_UltiPro Delivered Packages']/package[@name='Open Enrollment Reporting Package']/agentDefinition[@name='Op

In [131]:
#Convert XML results to JSON
o = xmltodict.parse(response.text)
json_data = json.dumps(o)

In [132]:
#Turn the JSON data into a Python dictionary
a_dict = json.loads(json_data)
a_dict

{'s:Envelope': {'@xmlns:s': 'http://www.w3.org/2003/05/soap-envelope',
  '@xmlns:a': 'http://www.w3.org/2005/08/addressing',
  's:Header': {'a:Action': {'@s:mustUnderstand': '1',
    '#text': 'http://www.ultipro.com/dataservices/bidata/2/IBIDataService/GetReportListResponse'}},
  's:Body': {'GetReportListResponse': {'@xmlns': 'http://www.ultipro.com/dataservices/bidata/2',
    'GetReportListResult': {'@xmlns:i': 'http://www.w3.org/2001/XMLSchema-instance',
     'Reports': {'Report': [{'ReportName': '_Current Employee Deductions and Benefits',
        'ReportPath': "/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/report[@name='_Current Employee Deductions and Benefits']"},
       {'ReportName': '2013-03-05T21:47:40.188Z',
        'ReportPath': "/content/folder[@name='UltiPro BI Content']/folder[@name='UltiPro BI for Core HR and Payroll']/folder[@name='_UltiPro Delivered Packages']/package[@name='Open Enrollment Reporting

In [133]:
#Index the Python dictionary 
emp_dict = a_dict['s:Envelope']['s:Body']['GetReportListResponse']['GetReportListResult']['Reports']['Report']
emp_dict

[{'ReportName': '_Current Employee Deductions and Benefits',
  'ReportPath': "/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/report[@name='_Current Employee Deductions and Benefits']"},
 {'ReportName': '2013-03-05T21:47:40.188Z',
  'ReportPath': "/content/folder[@name='UltiPro BI Content']/folder[@name='UltiPro BI for Core HR and Payroll']/folder[@name='_UltiPro Delivered Packages']/package[@name='Open Enrollment Reporting Package']/agentDefinition[@name='Open Enrollment Session Available - NEW']/report[@name='2013-03-05T21:47:40.188Z']"},
 {'ReportName': '2020-10-26T18:25:32.678Z',
  'ReportPath': "/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Customs']/agentDefinition[@name='Benefit Eligibility Alert']/report[@name='2020-10-26T18:25:32.678Z']"},
 {'ReportName': '2021 Allocation Report (1)',
  'ReportPath': "/content/folder[@name='zzzCo

In [134]:
#Normalize the dictionary, and convert into a pandas dataframe
df = pd.DataFrame.from_dict(pd.json_normalize(emp_dict), orient='columns')

df

,ReportName,ReportPath
0,_Current Employee Deductions and Benefits,/content/folder[@name='zzzCompany Folders']/fo...
1,2013-03-05T21:47:40.188Z,/content/folder[@name='UltiPro BI Content']/fo...
2,2020-10-26T18:25:32.678Z,/content/folder[@name='zzzCompany Folders']/fo...
3,2021 Allocation Report (1),/content/folder[@name='zzzCompany Folders']/fo...
4,401k Allocation Report V1,/content/folder[@name='zzzCompany Folders']/fo...
...,...,...
72,Test didactions,/content/folder[@name='zzzCompany Folders']/fo...
73,Test enrollments,/content/folder[@name='zzzCompany Folders']/fo...
74,UNUM Benefit Billing By Provider Report,/content/folder[@name='zzzCompany Folders']/fo...
75,Active Benefits (1),/content/folder[@name='zzzCompany Folders']/fo...


In [135]:
#View report names as a list
list(df['ReportName'])

['_Current Employee Deductions and Benefits',
 '2013-03-05T21:47:40.188Z',
 '2020-10-26T18:25:32.678Z',
 '2021 Allocation Report (1)',
 '401k Allocation Report V1',
 '401k Allocation Report V2',
 '401k Allocation Report V2_UKG',
 '401k Allocation Report V3_UKG',
 '401k Allocation Report V3_UKG v2',
 '401k Allocation Report V3_UKG v4',
 '5 Level Supervisors',
 'Active EE_DD List',
 'Allocated Hours FTE by Date',
 'Allocated Hours FTE by Month',
 'Annual Earnings',
 'Audit Report with Time vs Pay Matrix Tables',
 'Comp Project V3',
 'Configuration Validation',
 'Counts by Location',
 'Current Employee Deductions and Benefits',
 "David's Report",
 "David's Report V2",
 'Deduction list',
 'Deduction Pay History By Deduction For EmployeesUKG',
 'Deductions in Arrears V2',
 'Deferred Comp Wages and Hours_UKG',
 'Dependents by Employee and Plan test',
 'Document Category Report',
 'DSHS_New Hires',
 'Earning Codes Table',
 'Earnings Pay History By Earning Code For Employees_ULTI',
 'Earnings_

## Step 3: Use the ExecuteReport endpoint to get the report key

In [136]:
body = """   
  <s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing">

  <s:Header>

<a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/ExecuteReport</a:Action>

<a:To s:mustUnderstand="1">https://servicehost/services/BiDataService</a:To>

</s:Header>

  <s:Body>

  <ExecuteReport xmlns="http://www.ultipro.com/dataservices/bidata/2">

  <request xmlns:i="http://www.w3.org/2001/XMLSchema-instance">

<ReportPath>/content/folder[@name='zzzCompany Folders']/folder[@name='Proliance Surgeons, Inc., P.S.']/folder[@name='UltiPro']/folder[@name='Export to SQL']/report[@name='Pre-Check Deduction']</ReportPath>

  <ReportParameters>

  <ReportParameter>

<Name>Month</Name>

<Value>1</Value>

<Required>false</Required>

<DataType>xsdDouble</DataType>

<MultiValued>true</MultiValued>

</ReportParameter>

</ReportParameters>

</request>

  <context xmlns:i="http://www.w3.org/2001/XMLSchema-instance">

<ServiceId>3ccd26d5-94b2-4c0f-b88a-5ccfbe3d7fb0</ServiceId>

<ClientAccessKey>PLAB4</ClientAccessKey>

<Token>""" + token + """</Token>

<Status>Ok</Status>

<StatusMessage i:nil="true" />

<InstanceKey>""" + instance_key + """</InstanceKey>

</context>

</ExecuteReport>

</s:Body>

</s:Envelope> """

In [137]:
#Get the report key
response = requests.post(url, headers=headers, data=body)
print (response)
print (response.text)

<Response [200]>
<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bidata/2/IBIDataService/ExecuteReportResponse</a:Action></s:Header><s:Body><ExecuteReportResponse xmlns="http://www.ultipro.com/dataservices/bidata/2"><ExecuteReportResult xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><ReportKey>982fea7a-f828-4ec2-8b0c-27511ff120bd</ReportKey><ReportRetrievalUri>https://service5.ultipro.com/services/BIStreamingService</ReportRetrievalUri><Status>Success</Status><StatusMessage i:nil="true"/></ExecuteReportResult></ExecuteReportResponse></s:Body></s:Envelope>


In [138]:
#Write the response to a file
open("file.xml", "w").write(response.text)

687

In [139]:
#Open the file as an ElementTree object
tree = ET.parse('file.xml')
root = tree.getroot()

In [140]:
#Show all the elements in the tree
[elem.tag for elem in root.iter()]

['{http://www.w3.org/2003/05/soap-envelope}Envelope',
 '{http://www.w3.org/2003/05/soap-envelope}Header',
 '{http://www.w3.org/2005/08/addressing}Action',
 '{http://www.w3.org/2003/05/soap-envelope}Body',
 '{http://www.ultipro.com/dataservices/bidata/2}ExecuteReportResponse',
 '{http://www.ultipro.com/dataservices/bidata/2}ExecuteReportResult',
 '{http://www.ultipro.com/dataservices/bidata/2}ReportKey',
 '{http://www.ultipro.com/dataservices/bidata/2}ReportRetrievalUri',
 '{http://www.ultipro.com/dataservices/bidata/2}Status',
 '{http://www.ultipro.com/dataservices/bidata/2}StatusMessage']

In [141]:
#Get the report key from the "ReportKey" element in the tree
for each in root.iter('{http://www.ultipro.com/dataservices/bidata/2}ReportKey'):
    report_key = each.text

In [142]:
#View the report key
print(report_key)

982fea7a-f828-4ec2-8b0c-27511ff120bd


# Step 4: Use the RetrieveReport method to retrieve the report

In [143]:
#Our endpoint
url="https://service5.ultipro.com/services/BIStreamingService"

#The below headers variable specifies that we are sending the request to a SOAP API
headers = {"content-type": "application/soap+xml", 
           "charset": "utf-8",
           "SOAPAction": "http://www.ultipro.com/dataservices/bistream/2/IBIStreamService/RetrieveReport"}

In [144]:
body = """<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing">
	<s:Header>
		<a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bistream/2/IBIStreamService/RetrieveReport</a:Action>
		<h:ReportKey xmlns:h="http://www.ultipro.com/dataservices/bistream/2" xmlns="http://www.ultipro.com/dataservices/bistream/2">""" + report_key + """</h:ReportKey>
		<a:MessageID>urn:uuid:8332571f-3a89-4442-ac80-27195b397b4d</a:MessageID>
		<a:ReplyTo>
			<a:Address>http://www.w3.org/2005/08/addressing/anonymous</a:Address>
		</a:ReplyTo>
		<a:To s:mustUnderstand="1">https://service5.ultipro.com/services/BIStreamingService</a:To>
	</s:Header>
	<s:Body xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema">
		<RetrieveReportRequest xmlns="http://www.ultipro.com/dataservices/bistream/2"/>
	</s:Body>
</s:Envelope>"""

In [145]:
#Retrieve the report
response = requests.post(url, headers=headers, data=body)
print (response)
print (response.text)

<Response [200]>
<s:Envelope xmlns:s="http://www.w3.org/2003/05/soap-envelope" xmlns:a="http://www.w3.org/2005/08/addressing"><s:Header><a:Action s:mustUnderstand="1">http://www.ultipro.com/dataservices/bistream/2/IBIStreamService/RetrieveReportResponse</a:Action><h:Status xmlns:h="http://www.ultipro.com/dataservices/bistream/2">Completed</h:Status><h:StatusMessage i:nil="true" xmlns:h="http://www.ultipro.com/dataservices/bistream/2" xmlns:i="http://www.w3.org/2001/XMLSchema-instance"/><a:RelatesTo>urn:uuid:8332571f-3a89-4442-ac80-27195b397b4d</a:RelatesTo></s:Header><s:Body><StreamReportResponse xmlns="http://www.ultipro.com/dataservices/bistream/2"><ReportStream>PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPGRhdGFzZXQgIHhtbG5zPSJodHRwOi8vZGV2ZWxvcGVyLmNvZ25vcy5jb20vc2NoZW1hcy94bWxkYXRhLzEvIiAgeG1sbnM6eHM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDEvWE1MU2NoZW1hLWluc3RhbmNlIj4KPCEtLQo8ZGF0YXNldAogICAgeG1sbnM9Imh0dHA6Ly9kZXZlbG9wZXIuY29nbm9zLmNvbS9zY2hlbWFzL3htbGRhdGEvMS8iCiAgICB4bWxuczp4cz0

In [146]:
#Write the response to a file
open("file.xml", "w").write(response.text)

2948

In [147]:
#Open the file as an ElementTree object
tree = ET.parse('file.xml')
root = tree.getroot()

In [148]:
#Show all the elements in the tree
[elem.tag for elem in root.iter()]

['{http://www.w3.org/2003/05/soap-envelope}Envelope',
 '{http://www.w3.org/2003/05/soap-envelope}Header',
 '{http://www.w3.org/2005/08/addressing}Action',
 '{http://www.ultipro.com/dataservices/bistream/2}Status',
 '{http://www.ultipro.com/dataservices/bistream/2}StatusMessage',
 '{http://www.w3.org/2005/08/addressing}RelatesTo',
 '{http://www.w3.org/2003/05/soap-envelope}Body',
 '{http://www.ultipro.com/dataservices/bistream/2}StreamReportResponse',
 '{http://www.ultipro.com/dataservices/bistream/2}ReportStream']

In [149]:
#Get the token from the "Token" element in the tree
for each in root.iter('{http://www.ultipro.com/dataservices/bistream/2}ReportStream'):
    encoded_report = each.text 

In [150]:
#View the encoded report
encoded_report

'PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0idXRmLTgiPz4KPGRhdGFzZXQgIHhtbG5zPSJodHRwOi8vZGV2ZWxvcGVyLmNvZ25vcy5jb20vc2NoZW1hcy94bWxkYXRhLzEvIiAgeG1sbnM6eHM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDEvWE1MU2NoZW1hLWluc3RhbmNlIj4KPCEtLQo8ZGF0YXNldAogICAgeG1sbnM9Imh0dHA6Ly9kZXZlbG9wZXIuY29nbm9zLmNvbS9zY2hlbWFzL3htbGRhdGEvMS8iCiAgICB4bWxuczp4cz0iaHR0cDovL3d3dy53My5vcmcvMjAwMS9YTUxTY2hlbWEtaW5zdGFuY2UiCiAgICB4czpzY2hlbWFMb2NhdGlvbj0iaHR0cDovL2RldmVsb3Blci5jb2dub3MuY29tL3NjaGVtYXMveG1sZGF0YS8xLyB4bWxkYXRhLnhzZCIKPgotLT4KICAgIDxtZXRhZGF0YT4KICAgICAgICAgIDxpdGVtIG5hbWU9IlBlcmlvZCBDb250cm9sIERhdGUiIHR5cGU9InhzOmRhdGVUaW1lIi8+CiAgICAgICAgICA8aXRlbSBuYW1lPSJFbXBsb3llZSBOdW1iZXIiIHR5cGU9InhzOnN0cmluZyIgbGVuZ3RoPSIyMCIvPgogICAgICAgICAgPGl0ZW0gbmFtZT0iRGVkdWN0aW9uL0JlbmVmaXQgQ29kZSIgdHlwZT0ieHM6c3RyaW5nIiBsZW5ndGg9IjEyIi8+CiAgICAgICAgICA8aXRlbSBuYW1lPSJEZWR1Y3Rpb24vQmVuZWZpdCAoU3R1YikiIHR5cGU9InhzOnN0cmluZyIgbGVuZ3RoPSIzMiIvPgogICAgICAgICAgPGl0ZW0gbmFtZT0iQ3VycmVudCBBbW91bnQgKEVtcGxveWVlKSIgdHlwZT0ieHM6ZGVjaW1hbCI

The report is encoded in base 64, so we  need to decode it.

In [151]:
decoded = base64.b64decode(encoded_report).decode('utf-8')
decoded

'<?xml version="1.0" encoding="utf-8"?>\n<dataset  xmlns="http://developer.cognos.com/schemas/xmldata/1/"  xmlns:xs="http://www.w3.org/2001/XMLSchema-instance">\n<!--\n<dataset\n    xmlns="http://developer.cognos.com/schemas/xmldata/1/"\n    xmlns:xs="http://www.w3.org/2001/XMLSchema-instance"\n    xs:schemaLocation="http://developer.cognos.com/schemas/xmldata/1/ xmldata.xsd"\n>\n-->\n    <metadata>\n          <item name="Period Control Date" type="xs:dateTime"/>\n          <item name="Employee Number" type="xs:string" length="20"/>\n          <item name="Deduction/Benefit Code" type="xs:string" length="12"/>\n          <item name="Deduction/Benefit (Stub)" type="xs:string" length="32"/>\n          <item name="Current Amount (Employee)" type="xs:decimal" scale="4" precision="19"/>\n          <item name="Current Amount (Employer)" type="xs:decimal" scale="4" precision="19"/>\n    </metadata>\n    <data>\n        <row>\n            <value>2022-11-25T00:00:00</value>\n            <value>0

In [152]:
#Convert XML results to JSON
o = xmltodict.parse(decoded)
json_data = json.dumps(o)
json_data

'{"dataset": {"@xmlns": "http://developer.cognos.com/schemas/xmldata/1/", "@xmlns:xs": "http://www.w3.org/2001/XMLSchema-instance", "metadata": {"item": [{"@name": "Period Control Date", "@type": "xs:dateTime"}, {"@name": "Employee Number", "@type": "xs:string", "@length": "20"}, {"@name": "Deduction/Benefit Code", "@type": "xs:string", "@length": "12"}, {"@name": "Deduction/Benefit (Stub)", "@type": "xs:string", "@length": "32"}, {"@name": "Current Amount (Employee)", "@type": "xs:decimal", "@scale": "4", "@precision": "19"}, {"@name": "Current Amount (Employer)", "@type": "xs:decimal", "@scale": "4", "@precision": "19"}]}, "data": {"row": [{"value": ["2022-11-25T00:00:00", "001000827", "401K", "401k", "755.95", "755.95"]}, {"value": ["2022-11-25T00:00:00", "001006748", "OFFC", "Off-Cycle Check", "0", "0"]}, {"value": ["2022-11-25T00:00:00", "001007611", "OFFC", "Off-Cycle Check", "0", "0"]}]}}}'

In [153]:
#Turn the JSON data into a Python dictionary
a_dict = json.loads(json_data)
a_dict

{'dataset': {'@xmlns': 'http://developer.cognos.com/schemas/xmldata/1/',
  '@xmlns:xs': 'http://www.w3.org/2001/XMLSchema-instance',
  'metadata': {'item': [{'@name': 'Period Control Date',
     '@type': 'xs:dateTime'},
    {'@name': 'Employee Number', '@type': 'xs:string', '@length': '20'},
    {'@name': 'Deduction/Benefit Code', '@type': 'xs:string', '@length': '12'},
    {'@name': 'Deduction/Benefit (Stub)',
     '@type': 'xs:string',
     '@length': '32'},
    {'@name': 'Current Amount (Employee)',
     '@type': 'xs:decimal',
     '@scale': '4',
     '@precision': '19'},
    {'@name': 'Current Amount (Employer)',
     '@type': 'xs:decimal',
     '@scale': '4',
     '@precision': '19'}]},
  'data': {'row': [{'value': ['2022-11-25T00:00:00',
      '001000827',
      '401K',
      '401k',
      '755.95',
      '755.95']},
    {'value': ['2022-11-25T00:00:00',
      '001006748',
      'OFFC',
      'Off-Cycle Check',
      '0',
      '0']},
    {'value': ['2022-11-25T00:00:00',
      '

In [154]:
#Index the Python dictionary so we can retrieve the data
report_dict = a_dict['dataset']['data']['row']
report_dict

[{'value': ['2022-11-25T00:00:00',
   '001000827',
   '401K',
   '401k',
   '755.95',
   '755.95']},
 {'value': ['2022-11-25T00:00:00',
   '001006748',
   'OFFC',
   'Off-Cycle Check',
   '0',
   '0']},
 {'value': ['2022-11-25T00:00:00',
   '001007611',
   'OFFC',
   'Off-Cycle Check',
   '0',
   '0']}]

In [155]:
#Put the data into a list of lists
list_o_lst = []
for each in report_dict:
    list_o_lst.append(each['value'])

print(list_o_lst)

[['2022-11-25T00:00:00', '001000827', '401K', '401k', '755.95', '755.95'], ['2022-11-25T00:00:00', '001006748', 'OFFC', 'Off-Cycle Check', '0', '0'], ['2022-11-25T00:00:00', '001007611', 'OFFC', 'Off-Cycle Check', '0', '0']]


In [156]:
#Turn the list of lists into a dataframe
df = pd.DataFrame(list_o_lst)
df

,0,1,2,3,4,5
0,2022-11-25T00:00:00,001000827,401K,401k,755.95,755.95
1,2022-11-25T00:00:00,001006748,OFFC,Off-Cycle Check,0,0
2,2022-11-25T00:00:00,001007611,OFFC,Off-Cycle Check,0,0


In [157]:
#Rename the columns in the dataframe
df.rename(columns={0: 'Date', 1: 'Employee Number', 2: 'Deduction/Benefit Code', 3: 'Deduction/Benefit (Stub)', 4:'Current Amount (Employee)', 5: 'Current Amount (Employer)'}, inplace=True)
df

,Date,Employee Number,Deduction/Benefit Code,Deduction/Benefit (Stub),Current Amount (Employee),Current Amount (Employer)
0,2022-11-25T00:00:00,001000827,401K,401k,755.95,755.95
1,2022-11-25T00:00:00,001006748,OFFC,Off-Cycle Check,0,0
2,2022-11-25T00:00:00,001007611,OFFC,Off-Cycle Check,0,0


In [158]:
#Clean date column
df['Date'] = df['Date'].str.replace('T', ' ')

In [160]:
#Export the dataframe as a CSV file
#df.to_csv(r'C:\Users\zanarkin\Documents\Data Team\exported.csv')

# Step 5: Insert the data into BiSQL2

In [161]:
import pyodbc
import sqlalchemy as sal
from sqlalchemy import create_engine


In [162]:
#Create SQL alchemy engine
engine = sal.create_engine('mssql+pyodbc://DC-BISQL01/BIData2?driver=SQL Server?Trusted_Connection=yes')

In [163]:
conn = engine.connect()

#Create our connection object
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DC-BISQL01;'
                      'Database=BIData2;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [164]:
#See a sample from dbo.STG_PreCheck_Deduction table to test our database connection
#df = pd.read_sql_query('SELECT top 2 * FROM dbo.STG_PreCheck_Deduction', engine)
#df

In [165]:
#Drop table if it exists already
engine.execute('DROP TABLE ZZ_TEST_PreCheck_Deduction')

In [166]:
#Create a table in BiData2
engine.execute('CREATE TABLE ZZ_TEST_PreCheck_Deduction(period_control_date DATETIME,employee_number NVARCHAR(255),deduction_benefit_code NVARCHAR(255),current_amount_employee NVARCHAR(255),current_amount_employer NVARCHAR(255))')

In [167]:
#Add the data to our newly created table
df.to_sql('ZZ_TEST_PreCheck_Deduction', engine, if_exists='replace')

In [168]:
#Commit changes to BiData2
#conn.commit()

AttributeError: 'Connection' object has no attribute 'commit'

In [ ]:
#Close the connection to BiData2
conn.close()

In [ ]:
#Close the connection to the API